In [1]:
!pip install num2words

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 KB 30.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=e25899965d6b51302ba2f2e6cef80034d6e767837e8f2836d584ad37e331c11a
  Stored in directory: /tmp/pip-ephem-wheel-cache-fhnor03l/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt


In [2]:
from tqdm.autonotebook import tqdm
from num2words import num2words
import copy
import itertools


/tmp/ipykernel_26/660108839.py:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
class Word:
    def __init__(self, word, word_type):
        self.value = word # or node
        self.word_type = word_type
        
class Template:
    def __init__(self, rus, latex):
        rus = TemplateLang(rus)
        latex = TemplateLang(latex)
        
        self.langs = [rus, latex]
        
    def get_langs(self): return self.langs

    def get_vars_count(self):
        return self.langs[0].vars_count()
    
    def __repr__(self):
        return self.langs[0].get_words()[0].value + self.langs[0].get_words()[1].value + self.langs[0].get_words()[2].value

class TemplateLang:
    def __init__(self, pattern):
        self.nodes = []
        self.vars = 0
        self.prepare_pattern(pattern)
    
    def prepare_pattern(self, pattern):
        save_var = False
        for char in pattern:
            if char == '*':
                self.vars += 1
                save_var = True
            elif save_var:
                self.add_word(Word(int(char), 'var'))
                save_var = False
            else:
                self.add_word(Word(char, 'word'))
    
    def vars_count(self):
        return self.vars
    
    def add_word(self, node):
        self.nodes += [node]
        
    def get_words(self):
        return self.nodes

class Node:
    def __init__(self, template1, template2):
        self.template = Template(template1, template2)
        self.childs = []
        #self.parent = None
        
    def add_child(self, child):
        self.childs += [child]
        #child.parent = self
    
    #def add_child_str(self, child):
    #    self.childs_str += [child]
    
    #def find_source(self):
    #    node = self
    #    while(node.parent != None):
    #        node = node.parent
    #    return node
    
    def get_count_of_childs(self):
        return self.template.get_vars_count()
    
    def get_str(self, args):
        rus,latex = self.template.get_langs()
        rus = rus.get_words()
        latex = latex.get_words()
        
        self.childs_str = args
        #print(self.childs_str)
        
        rus_new = str()
        child_num = 0
        keys = {}
        for word in rus:
            if word.word_type == 'var':
                if self.childs_str == []:
                    rus_new += '#'
                else:
                    child = self.childs_str[child_num]
                    child_num += 1
                    
                    keys[word.value] = child
                    #print(child[0])
                    string = child[0][0]
                    if len(string.split(' '))>1 and child_num<len(args): rus_new += 'скобка открывается '
                    # add more complex exceptions when you shouldnt place bracket
                    rus_new += string
                    if len(string.split(' '))>1 and child_num<len(args): rus_new += ' скобка закрывается'
            else:
                rus_new += word.value
        
        latex_new = str()
        child_num = 0
        for word in latex:
            if word.word_type == 'var':
                if self.childs_str == []:
                    latex_new += '#'
                else:
                    child = keys[word.value]
                    child_num += 1
                    
                    latex_new += child[1][0]
            else: latex_new += word.value
        #print(rus_new, latex_new)
        return (rus_new, latex_new)

In [4]:
class Function:
    def __init__(self, temp1, temp2):
        self.node = Node(temp1,temp2)
        self.arguments = []
        self.name = ''
        self.args_count = self.node.get_count_of_childs()
    
    def get_args_count(self):
        return self.args_count
    
    def __repr__(self):
        return self.name + ' ' + str(self.arguments)
        
    def add_argument(self, args_variation):
        self.arguments += [args_variation]
        
    def get_strs(self):
        results = []
        args = self.arguments
        
        args_flatten = []
        for arg_num in args:
            this_arg = []
            for arg_i in arg_num:
                this_arg += [arg_i.get_strs()] #all_variations
            args_flatten += [this_arg]
        #print(args_flatten)
        # funcs
        results_rus = []
        results_latex = []
        
        data = itertools.product(*args_flatten)
        str_i = ''
        for func in data:
            str_i = self.node.get_str(func)
            results_rus += [str_i[0]]
            results_latex += [str_i[1]]
        return (results_rus, results_latex)


In [5]:
#F_full = [
#    Function('икс', 'x'),
#     Function('игрек', 'y'),
#    ]

#F_full = [
#    Function('дэ'     ,'d'  ),
#    Function('е'      ,'e'  ),
#    Function('логарифм *1 по основанию *2', 'log_{*2}{*1}'),
#]
#
F_1 = [
    Function('логарифм *1 по основанию *2', 'log_{*2}{*1}'),
    
    Function('*1 плюс *2','*1 + *2'),
    Function('*1 минус *2','*1 - *2'),
    Function('*1 умножить на *2','*1 *2'),
    Function('*1 разделить на *2','*1 / *2'),
    Function('*1 *2','*1 *2'),
    Function('предел *1 при *2 стремящийся к *3','\lim_{*2\to*3}{*1}'),
    Function('произведение *1 от *2 равно *3 до *4','\prod_{*2=*3}^{*4} *1'),
    Function('сумма *1 от *2 равно *3 до *4','\sum_{*2=*3}^{*4} *1'),
    Function('интеграл *1 от *2 до *3 по дэ *4','\int_{*2}^{*3} *1 \,d*4'),
    Function('*1 в степени *2','*1^*2'),
    Function('*1 квадрат','*1^2'),
    Function('*1 куб','*1^3'),
    Function('*1 в *2 степени','*1^*2'),
    Function('корень из *1','\sqrt{*1}'),
    Function('дробь *1 разделить на *2','\frac{*1}{*2}'),
    
    Function('синус *1','\sin{*1}'),
    Function('косинус *1','\cos{*1}'),
    Function('тангенс *1','\tg{*1}'),
    Function('котангенс *1','\ctg{*1}'),
    Function('секанс *1','\sec{*1}'),
    Function('косеканс *1','\cosec{*1}'),
]

F_2 = [
    Function('а'      ,'a'  ),
    Function('бэ'     ,'b'  ),
    Function('цэ'     ,'c'  ),
    Function('дэ'     ,'d'  ),
    Function('е'      ,'e'  ),
    Function('эф'     ,'f'  ),
    Function('джи'    ,'g'  ),
    Function('аш'     ,'h'  ),
    Function('и'      ,'i'  ),
    Function('жи'     ,'j'  ),
    Function('ка'     ,'k'  ),
    Function('эл'     ,'l'  ),
    Function('эм'     ,'m'  ),
    Function('эн'     ,'n'  ),
    Function('о'      ,'o'  ),
    Function('пэ'     ,'p'  ),
    Function('ку'     ,'q'  ),
    Function('эр'     ,'r'  ),
    Function('эс'     ,'s'  ),
    Function('тэ'     ,'t'  ),
    Function('у'      ,'u'  ),
    Function('вэ'     ,'w'  ),
    Function('ви'     ,'v'  ),
    Function('икс'    ,'x'  ),
    Function('игрек'  ,'y'  ),
    Function('зэт'    ,'z'  ),
    Function('альфа'  ,'α'  ),
    Function('бета'   ,'β'  ),
    Function('гамма'  ,'γ'  ),
    Function('дельта' ,'Δ'  ),
    Function('эпсилон','ε'  ),
    Function('дзета'  ,'ζ'  ),
    Function('эта'    ,'η'  ),
    Function('тета'   ,'θ'  ),
    Function('лямбда' ,'λ'  ),
    Function('мю'     ,'μ'  ),
    Function('ню'     ,'ν'  ),
    Function('кси'    ,'ξ'  ),
    Function('омикрон','ο'  ),
    Function('пи'     ,'π'  ),
    Function('ро'     ,'ρ'  ),
    Function('сигма'  ,'σ'  ),
    Function('тау'    ,'τ'  ),
    Function('ипсилон','υ'  ),
    Function('фи'     ,'φ'  ),
    Function('хи'     ,'χ'  ),
    Function('пси'    ,'ψ'  ),
    Function('омега'  ,'ω'  )
]

In [6]:

#F_full = []
for i in range(10000):
    F_2 += [Function(num2words(i, lang='ru'), str(i))]


In [8]:
def function_filling(f, deep, deep_max):
    
    add_args = []
    for i in range(f.get_args_count()):
        if deep==deep_max: F_add = F_0
        else: F_add = F_full
        
        to_add = []
        for to_fill in F_add:
            to_fill = copy.deepcopy(to_fill)
            new_func = function_filling(to_fill, deep+1, deep_max)
            to_add += [new_func]
        add_args += [to_add]

    for A in add_args:
        f.add_argument(A)
    return f

R_rus = []
R_latex = []
    
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

for F_i in tqdm(list(chunks(F_2, 1))):
    F_full = copy.deepcopy(F_1 + F_i)
    F_0 = [f for f in F_full if f.get_args_count()==0]
    
    for func in F_full:
        new = function_filling(func, 0, 0)
        strs = new.get_strs()
        
        R_rus += strs[0]
        R_latex += strs[1]
    
#print(R_rus)
#print(R_latex)
#print(list(itertools.product(*new.arguments)))
#for f in F_full:
#    f

  0%|          | 0/10048 [00:00<?, ?it/s]

In [6]:
#strs

(['логарифм дэ по основанию дэ',
  'логарифм дэ по основанию е',
  'логарифм дэ по основанию логарифм дэ по основанию дэ',
  'логарифм е по основанию дэ',
  'логарифм е по основанию е',
  'логарифм е по основанию логарифм дэ по основанию дэ',
  'логарифм скобка открывается логарифм дэ по основанию дэ скобка закрывается по основанию дэ',
  'логарифм скобка открывается логарифм дэ по основанию дэ скобка закрывается по основанию е',
  'логарифм скобка открывается логарифм дэ по основанию дэ скобка закрывается по основанию логарифм дэ по основанию дэ'],
 ['log_{d}{d}',
  'log_{e}{d}',
  'log_{log_{d}{d}}{d}',
  'log_{d}{e}',
  'log_{e}{e}',
  'log_{log_{d}{d}}{e}',
  'log_{d}{log_{d}{d}}',
  'log_{e}{log_{d}{d}}',
  'log_{log_{d}{d}}{log_{d}{d}}'])

In [9]:
res = []
for i,val in enumerate(R_rus):
    res += [[R_rus[i], R_latex[i]]]
#res

In [10]:
len(res)

231104

In [11]:
import pandas as pd

df = pd.DataFrame(res, columns=['rus','latex'])
df.to_csv('df_GAN.csv',index=False)